<a href="https://colab.research.google.com/github/JakobHafke/RL_Task_7/blob/main/RL_Assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Robot Learning

## Assignment 7

Solutions are due on 03.06.2025 before the lecture.

### Introduction

On this assignment sheet, we will use an environment from [OpenAi Gym](https://github.com/openai/gym). The setup requires some further packages. Since this installation is not trivial, and we could only test it in our setup, we strongly recommend that you execute the cells in this notebook in [Google Colab](https://research.google.com/colaboratory/). You should find a button which opens this file directly in Colab at the top of this notebook. If not, you can simply import the .ipynb manually.

If you have started your Colab session and are ready to proceed, uncomment the three lines in the code cell below. They will install everything required to simulate the Gym environment. If prompted to restart your runtime, do so, but you don't have to repeat the installation unless you delete your runtime.

**Warning: This is unlikely to work on your own computer, and might even mess up your system! Please only use the following lines in Colab. If you insist on using your own machine, please refer to installation instructions for Gym, torch and the box2d environments for your system.**

In [1]:
!apt-get -qq install xvfb x11-utils &> /dev/null
!pip install ufal.pybox2d --quiet &> /dev/null
!pip install pyvirtualdisplay moviepy pyglet PyOpenGL-accelerate --quiet &> /dev/null
!pip install numpy==1.23.5 matplotlib==3.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 122.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.7 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you ha

The following cell imports packages required for the task.

In [1]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from pyvirtualdisplay import Display
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import torch
import torch.nn as nn
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.11/dist-packages/moviepy/config_defaults.py:1: DeprecationWarning: invalid escape sequence '\P'
  """
/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: DeprecationWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
  from scipy.ndimage.filters import sobel

  if event.key is 'enter':



## Task 7.1)

Your agent is in state $s_t$ and has the $Q$-values $Q(s_t,a) = [Q(s_t,a_0), Q(s_t,a_1), Q(s_t,a_2), Q(s_t,a_3)] = [5, 7, 3, 9]$. If the agent samples its action according to a probabilistic policy $\pi(s_t,a)$ which is created by softmax action selection from $Q(s_t,a)$, then what is the probability $Pr(a_3 | s_t)$ of taking action $a_3$ in state $s_t$?

<div style="text-align: right; font-weight:bold"> 3 Points </div>

$Pr(a_3|s_t) = \frac{e^{Q(s_t, a_3)}}{\sum_{i=0}^{3} e^{Q(s_t, a_i)}} = \frac{e^9}{e^5 + e^7 + e^3 + e^9} \approx 0.865 $



## Task 7.2)

In this task, we will combine an actor-critic method like on the previous sheet with a policy gradient algorithm to control an agent in a challenging environment: the Lunar Lander.

In [2]:
# set up showing animations from the environment in Colab.
Display(visible=False).start()

Examine the code cell below. It has two distinct purposes:

* Showcase the agent-environment interaction for LunarLander-v2

* Show how you can capture frames from this environment to animate an episode afterwards.

Note that for training an agent in this environment, it is advisable to omit all code corresponding to the rendering. You can seperately render an episode of your agent's play afterwards.


In [8]:
# Name of the environment, if you are having problems you can switch to 'CartPole-v1', which is easier to solve.
ENV_NAME = 'LunarLander-v2' #'CartPole-v1'
# Dimension of the LunarLander state space. For 'CartPole-v1', use 4 instead
ENV_STATE_DIM =  8 # 4
# Lunar Lander has 4 discrete actions: [Do Nothing, Fire Left Booster, Fire Main Engine, Fire Right Booster], 'CartPole-v1' has 2
ENV_ACTION_DIM = 4 # 2
# If the agent reaches this score the task is seen as solved
SCORE_TO_SOLVE = 200 # 195
MAX_STEPS = None # 500
# Create the environment
env = gym.make(ENV_NAME)
# Reset the environment
state = env.reset() # state = [x, y, dx, dy, theta, dtheta, leg1_contact, leg2_contact]
# Track whether the episode is over
done = False
# List to append the frames produced by the environment renderer
frames = []
while not done:
  # Render current situation and append to frames
  frames.append(env.render('rgb_array'))
  # Select a random action
  action = env.action_space.sample()
  # Execute this action
  state, reward, done, info = env.step(action)
# Print the number of frames
print('Number of frames:', len(frames))
# Prevent the renderer from showing artifacts
plt.close()

Number of frames: 139


In [9]:
# Helper function to animate a list of frames as produced above
def visualize_trajectory(frames, fps=50):
  duration = int(len(frames) // fps + 1)
  fig, ax = plt.subplots()
  def make_frame(t, ind_max=len(frames)):
      ax.clear()
      ax.imshow(frames[min((int(fps*t),ind_max-1))])
      return mplfig_to_npimage(fig)
  plt.close()
  return VideoClip(make_frame, duration=duration)

In [10]:
# Get the animation from the frames of the played episode
animation = visualize_trajectory(frames)
# Show the animation
animation.ipython_display(fps=50, loop=True, autoplay=True)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


Probably, the random agent will destroy itself instead of landing between the two flags. We would like you to improve upon this. Below, you are given a neural net with learnable weights $\theta$ which takes an environment state $s_t$ as input and can output either a state value $V_{\theta}(s_t)$ or a probability distribution over the actions $\pi_{\theta}(s_t,a)$.

In [11]:
# If you feel like it, you can, but you do not need to adapt this
class DualNet(nn.Module):
    def __init__(self, state_dim=ENV_STATE_DIM, action_dim=ENV_ACTION_DIM, hidden_layer_dim=42):
        super(DualNet, self).__init__()
        # Create some layers to encode the input state
        self.layers = [nn.Linear(state_dim, hidden_layer_dim),
                       nn.LeakyReLU()]
        # Combine these layers into a net
        self.net = nn.Sequential(*self.layers)
        # Critic output layers to estimate V from the state encoding
        self.critic = nn.Sequential(*[nn.Linear(hidden_layer_dim,1)])
        # Actor output layers to estimate pi from the state encoding
        self.actor = nn.Sequential(*[nn.Linear(hidden_layer_dim,action_dim),
                                        nn.Softmax(dim=-1)])
    def forward(self, s, mode):
        # Convert input state to tensor
        device = next(self.net.parameters()).device
        x = torch.tensor(s).float().view(1,-1).to(device)
        # Encode state
        x = self.net(x)
        if mode == 'actor':
          # Return probability distribution over actions
          x = self.actor(x)
        else:
          # Return estimate of state value
          x = self.critic(x)
        return x.squeeze()

# Example usage:
# Create instance of the DualNet class
test_net = DualNet()
# Create a dummy state, round just for pretty printing
test_input = np.around(np.random.rand(ENV_STATE_DIM),2)
# Get the actor output
actor_out = test_net(test_input, mode='actor')
# Get the critic output
critic_out = test_net(test_input, mode='critic')
print('Dummy Input:', test_input)
print('Actor output:', actor_out)
print('Critic output:', critic_out)

Dummy Input: [0.89 0.59 0.83 0.21 0.81 0.87 0.24 0.38]
Actor output: tensor([0.2838, 0.2380, 0.2380, 0.2402], grad_fn=<SqueezeBackward0>)
Critic output: tensor(-0.0309, grad_fn=<SqueezeBackward0>)


Now to the task: Play episodes according to the following scheme:

* For each visited state $s_t$, store the output of the critic $V_\theta(s_t)$ in a list.

* Select an action $a_t$ by sampling from the distribution $\pi_{\theta}(s_t,a)$ output by the actor. In a list, store the log prob of the action: $l_t = log(\pi_{\theta}[s_t,a_t])$.

* Execute the action and observe the reward $r_{t+1}$ provided by the environment.

After each episode, use the stored rewards to calculate the Returns $R_t$ following each state $s_t$ using the discount factor $\gamma$. Next, calculate for each $t$

$$\delta_t = R_t - V_{\theta}(s_t)$$

Then, calculate the loss of the critic as

$$L_{critic} = 0.5 \sum_t \delta_t^2$$

and the loss of the actor using the log probs as

$$L_{actor} = \sum_t - l_t  \delta_t$$

and finally the total loss

$$L = L_{critic} + L_{actor}$$

Now, update the parameters $\theta$ using

$$\theta \leftarrow \theta + \alpha \nabla_{\theta}L$$

The Lunar Lander problem is considered solved when the agent achieves an average return of 200 over 100 episodes. Solve the problem, or play around 3000 episodes as described above and then report the average return of the final 100 episodes played. This means that you are welcome to preemptively stop training if the average return is sufficient. Then, play one more episode and animate it like in the example above. Use $\gamma = 0.99$ in your experiments. Save your final animation and place it into your sciebo folder along with this notebook. We recommend starting with a learning rate of $\alpha = 0.02$, but please note that this parameter might be sensitive to the specifics of your implementation. Solving the environment should not take longer than an hour, but we have seen implementations that completed the task in as little as 10 minutes.

### Hints:

Following tips might help you complete this task:

* You might need to convert your return $R_t$ to the correct datatype:

$$\delta_t = torch.tensor(R_t) - V_{\theta}(s_t)$$

* If you want to use numpy to sample from $\pi_{\theta}(s_t,a)$, you can get a numpy array by calling

$$\pi_{\theta}(s_t,a).detach().numpy()$$

* When you calculate the log probs $l_t$, preserve the torch gradient graph by using the torch function

$$l_t = torch.log(\pi_{\theta}[s_t,a_t])$$

* When you calculate $L_{actor}$, use $\delta_t.item()$ to ensure that the actor's loss does not directly influence the critic's gradients.

* It might be easier to solve the task for the Cart-Pole environment first, just change ENV_NAME, ENV_STATE_DIM and ENV_ACTION_DIM in one of the previous code blocks.

* Standardizing the Returns (zero mean and std 1) before calculating $\delta_t$ can boost performance.

* Below you are already given a rough structure for the algorithm. If you stick to it, torch will compute and apply $\nabla_{\theta}L$ for you!

<div style="text-align: right; font-weight:bold"> 10 +  3 (animation) Points </div>

In [24]:
# Complete this code or write your own!
# Create the environment
env = gym.make(ENV_NAME)
# Get the combined actor and critic model
net = DualNet()
# if using a gpu runtime DualNet().cuda() moves the policy to GPU
# in this case, use .cpu() on a tensor before converting to numpy
# Number of episodes to play, maybe use fewer at first
num_iter = 3000
# Learning rate for the parameter updates
alpha = 2e-2
# Discount factor
gamma = 0.99
# The optimizer will do the gradient updates for you
# It needs the trainable parameters and a learning rate
optimizer = torch.optim.RMSprop(net.parameters(), lr=alpha)
# feel free to experiment with other torch optimizers

# test_input = np.around(np.random.rand(ENV_STATE_DIM),2)
# actor_out = test_net(test_input, mode='actor')
# critic_out = test_net(test_input, mode='critic')

# This progress_bar is useful to know how far along the training is
progress_bar = tqdm(range(num_iter), total=num_iter, position=0, leave=True)
# For each episode (episode can be used like an int)
total_returns = []
for episode in progress_bar:
    # Reset the accumulated gradients of the model parameters
    optimizer.zero_grad()
    # Reset the environment and observe the starting state
    s = env.reset()
    done = 0
    # Collect the following terms during the episode
    rewards = []
    state_vals = []
    log_probs = []
    while not done:
        # During each Episode:
        # Evaluate the critic for s, store it
        state_val = net(s, mode='critic')
        state_vals.append(state_val)
        # Evaluate the actor for s
        pi = net(s, mode='actor')
        # Sample a from the distribution given by the actor
        a = torch.distributions.Categorical(probs=pi).sample()
        # Store log_prob of a
        log_prob = torch.log(pi[a])
        log_probs.append(log_prob)
        # Execute action a, observe next state, reward and done
        s, r, done, _ = env.step(a.item())
        # Store the reward
        rewards.append(r)

    # After each episode is done
    # Calculate the Returns from the episode. Use rewards and gamma
    R = 0
    returns = []
    for r in reversed(rewards):
        R = r + gamma * R
        returns.append(R)
    returns = list(reversed(returns))
    returns = torch.tensor(returns)

    # You might want to standardize the Returns
    returns = (returns - returns.mean()) / (returns.std())

    # Calculate delta_t, L_actor, L_critic
    state_vals = torch.stack(state_vals)
    log_probs = torch.stack(log_probs)

    delta_t = returns - state_vals

    L_critic = 0.5 * torch.sum(delta_t.pow(2))
    L_actor = torch.sum(-log_probs * delta_t.detach())

    L = L_critic + L_actor
    L.backward()
    optimizer.step()

    total_returns.append(np.sum(rewards))
    avg_reward = np.mean(rewards[-100:])

    # If you want your progress bar to print info, you can use the following template
    # How often to update info
    if episode % 10 == 0:
        # When to first update info
        if episode > 99:
            # List of strings containing info
            episode_summary = [
                "Episode {}".format(episode),
                "Reward: {:.1f}".format(avg_reward),
                "L_actor: {:.2f}".format(L_actor.item()),
                "L_critic: {:.2f}".format(L_critic.item())
            ]
            # Set progress bar
            progress_bar.set_description(" | ".join(episode_summary))

Episode 2990 | Reward: 1.5 | L_actor: 4.76 | L_critic: 66.10: 100%|██████████| 3000/3000 [25:50<00:00,  1.93it/s]


In [26]:
state = env.reset()
done = False
frames = []
while not done:
  frames.append(env.render('rgb_array'))
  pi = net(state, mode='actor')
  action = torch.distributions.Categorical(probs=pi).sample().item()
  # action = torch.argmax(pi).item()
  state, reward, done, info = env.step(action)
animation = visualize_trajectory(frames)
# animation.save('lunar_lander.mp4')
animation.ipython_display(fps=50, loop=True, autoplay=True)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


## Task 7.3)

In the previous task, you calculate the loss of the policy network as

$$L_{actor} = \sum_t - log(\pi_{\theta}[s_t,a_t])  \delta_t$$

with

$$\delta_t = R_t - V_{\theta}(s_t).$$

Give an intuitive explanation why minimizing this term leads to actions with a good outcome being more likely and actions with a bad outcome becoming less likely.

<div style="text-align: right; font-weight:bold"> 4 Points </div>

The delta determines wether the step the agent took increased or decreased the total value. A positive delta value means the step was a useful one with current knowledge about state values, a negative delta value means the step was not so good.
The log probs get very negative for small probabilities, which means that when the agent takes a very unlikely action the delta gets scaled by a large negative number. Should the delta be negative the agent learns to decrease this probability even further, when the delta is positive this probability gets boosted a lot in the net learning step.
On the other hand, for large probability actions the delta values get scaled with values close to zero. This makes sense, since when the agent takes a very likely action (extreme case: probability 1 -> log. prob. = 0) not much can be learned from this path and this step should not contribute much to the loss.
